In [22]:
import pandas as pd
from sklearn.preprocessing import RobustScaler, StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error , mean_squared_error , r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,GradientBoostingRegressor,VotingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

import pickle

import warnings
warnings.filterwarnings('ignore')

In [31]:
df_processed=pd.read_csv("../data/processed/df_copy_encoded.csv")
df_processed.head()

,year,customer_age,unit_cost,unit_price,cost,revenue,hour,profit,profit_margin,country_0,country_1,country_2,product_category_0,product_category_1,sub_category_0,sub_category_1,sub_category_2,sub_category_3,sub_category_4,customer_gender_M
0,2016.0,29.0,80.00,109.000000,80.0,109.0,0,29.0,0.266055,0,0,1,0,1,0,0,0,0,1,0
1,2016.0,29.0,24.50,28.500000,49.0,57.0,0,8.0,0.140351,0,0,1,1,0,0,0,0,1,0,0
2,2016.0,29.0,3.67,5.000000,11.0,15.0,0,4.0,0.266667,0,0,1,0,1,0,0,0,0,1,0
3,2016.0,29.0,87.50,116.500000,175.0,233.0,0,58.0,0.248927,0,0,1,0,1,0,0,0,0,1,0
4,2016.0,29.0,35.00,41.666667,105.0,125.0,0,20.0,0.160000,0,0,1,0,1,0,0,0,0,1,0


In [32]:
df_processed.drop('profit_margin',axis=1,inplace=True)
df_processed.drop('revenue',axis=1,inplace=True)

In [46]:
df_processed.drop('hour',axis=1,inplace=True)

In [47]:
x= df_processed.drop("profit", axis=1)
y= df_processed['profit']

In [49]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [50]:
x_train

,year,customer_age,unit_cost,unit_price,cost,country_0,country_1,country_2,product_category_0,product_category_1,sub_category_0,sub_category_1,sub_category_2,sub_category_3,sub_category_4,customer_gender_M
4103,2015.0,39.0,275.00,299.500000,550.0,0,1,1,1,0,0,0,1,1,1,0
10316,2015.0,53.0,184.67,240.666667,554.0,1,0,0,0,1,0,0,0,0,1,0
29297,2016.0,30.0,256.33,247.666667,769.0,0,0,1,1,1,0,0,1,0,1,1
7091,2016.0,38.0,210.00,266.000000,210.0,0,1,1,0,1,0,0,0,1,1,0
5642,2016.0,55.0,773.33,989.333333,2320.0,1,0,0,1,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20757,2015.0,28.0,171.00,135.000000,171.0,0,1,0,0,1,0,1,1,0,1,1
32103,2016.0,35.0,103.50,134.500000,207.0,0,1,1,1,0,0,1,0,1,0,1
30403,2015.0,23.0,300.00,338.500000,600.0,0,0,1,0,1,0,0,0,0,1,1
21243,2016.0,42.0,30.00,36.000000,60.0,0,0,1,0,1,0,1,1,0,1,0


In [51]:
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [52]:
x_train

array([[-1.14897632,  0.23789758, -0.15231997, ...,  1.36022603,
         0.35134364, -1.02280281],
       [-1.14897632,  1.50119545, -0.33718479, ..., -0.73517193,
         0.35134364, -1.02280281],
       [ 0.87033996, -0.57422248, -0.19052905, ..., -0.73517193,
         0.35134364,  0.97770556],
       ...,
       [-1.14897632, -1.20587141, -0.10115623, ..., -0.73517193,
         0.35134364,  0.97770556],
       [ 0.87033996,  0.50860427, -0.65372462, ..., -0.73517193,
         0.35134364, -1.02280281],
       [ 0.87033996,  0.50860427, -0.68100512, ..., -0.73517193,
         0.35134364,  0.97770556]])

In [53]:
models={
    'linear_reg':LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "knn": KNeighborsRegressor(n_neighbors=6),
    'SVR':SVR(C=10,kernel='linear'),
    'DT':DecisionTreeRegressor(),
    'RF':RandomForestRegressor(n_estimators=50,n_jobs=-1),
    'Bagging_classifier':BaggingRegressor(DecisionTreeRegressor(),n_estimators=5,n_jobs=-1),
    'xgboost':GradientBoostingRegressor(random_state=0),
 
}

In [54]:
for name,model in models.items():
    print('--------- ',name,'-------------')
    model.fit(x_train,y_train)
   
    y_train_pred= model.predict(x_train)
    y_test_pred= model.predict(x_test)
    print("Train")
    print("R2",r2_score(y_train, y_train_pred) )
    print("MSE",mean_squared_error(y_train, y_train_pred, squared= False ))
    print("MAE",mean_absolute_error(y_train,y_train_pred))
    print("Test")
    print("R2",r2_score(y_test, y_test_pred) )
    print("MSE",mean_squared_error(y_test, y_test_pred, squared= False ))
    print("MAE",mean_absolute_error(y_test,y_test_pred))
    pickle.dump(model,open("../models/"+name+"_model.h5",'wb'))
    
   
    print('-'*30)

---------  linear_reg -------------
Train
R2 0.8389589450146101
MSE 61.70553065173316
MAE 35.16683412628726
Test
R2 0.8314987615264025
MSE 61.40194191393045
MAE 35.226005652539946
------------------------------
---------  Ridge -------------
Train
R2 0.8389557753929677
MSE 61.70613789509439
MAE 35.187660496956006
Test
R2 0.8314637310074235
MSE 61.40832415221631
MAE 35.24741142956816
------------------------------
---------  Lasso -------------
Train
R2 0.8332220753550887
MSE 62.795000846375686
MAE 35.628150081322154
Test
R2 0.8253590814105425
MSE 62.51058473120361
MAE 35.75099830623318
------------------------------
---------  knn -------------
Train
R2 0.7928954317759955
MSE 69.97624390060001
MAE 36.27613252925551
Test
R2 0.6456973439069693
MSE 89.0364348526794
MAE 44.95541995692749
------------------------------
---------  SVR -------------
Train
R2 0.78578771521008
MSE 71.16688789972244
MAE 31.737681421997564
Test
R2 0.776016850242252
MSE 70.79266009862003
MAE 31.82214555683863
----

In [55]:
poly=PolynomialFeatures(degree=2)

x_train_poly=poly.fit_transform(x_train)
x_test_poly=poly.transform(x_test)


poly_regression_model=LinearRegression()

poly_regression_model.fit(x_train_poly,y_train)

poly_regression_model.score(x_train_poly,y_train)

y_train_pred= poly_regression_model.predict(x_train_poly)
y_test_pred= poly_regression_model.predict(x_test_poly)

print("Train")
print("R2",r2_score(y_train, y_train_pred) )
print("MSE",mean_squared_error(y_train, y_train_pred, squared= False ))
print("MAE",mean_absolute_error(y_train,y_train_pred))
print("Test")
print("R2",r2_score(y_test, y_test_pred) )
print("MSE",mean_squared_error(y_test, y_test_pred, squared= False ))
print("MAE",mean_absolute_error(y_test,y_test_pred))
pickle.dump(poly_regression_model,open("../models/"+"polynomial_regression"+"_model.h5",'wb'))


Train
R2 0.9794309261082366
MSE 22.052770465175666
MAE 12.900274888689824
Test
R2 0.976356100588853
MSE 23.000649999089177
MAE 13.247204931605573


In [56]:
y_train_pred

array([ 52.05175781, 150.64379883,   5.3416748 , ...,  86.40576172,
        13.30114746,  28.74438477])

In [57]:
x_train_poly[0]

array([ 1.00000000e+00, -1.14897632e+00,  2.37897583e-01, -1.52319966e-01,
       -1.70807787e-01, -3.70172298e-02, -4.19096002e-01,  1.41569974e+00,
        6.51443605e-01,  1.35851268e+00, -2.38777457e+00, -1.42311985e-01,
       -7.74255630e-01,  1.14068969e+00,  1.36022603e+00,  3.51343638e-01,
       -1.02280281e+00,  1.32014658e+00, -2.73338689e-01,  1.75012034e-01,
        1.96254102e-01,  4.25319205e-02,  4.81531381e-01, -1.62660548e+00,
       -7.48493275e-01, -1.56089889e+00,  2.74349644e+00,  1.63513100e-01,
        8.89601383e-01, -1.31062544e+00, -1.56286750e+00, -4.03685520e-01,
        1.17517621e+00,  5.65952599e-02, -3.62365517e-02, -4.06347596e-02,
       -8.80630950e-03, -9.97019257e-02,  3.36791546e-01,  1.54976859e-01,
        3.23186882e-01, -5.68045799e-01, -3.38556771e-02, -1.84193543e-01,
        2.71367320e-01,  3.23594485e-01,  8.35838022e-02, -2.43322317e-01,
        2.32013721e-02,  2.60174363e-02,  5.63846319e-03,  6.38366887e-02,
       -2.15639337e-01, -

### Polynomial regression of degree 2 is the best model as it have 97.6 r2 score in test and 97.9 in train . There is a bit overfitting but it can be accepted as it is not too much.
### The other models either of lower r2 score or of higher overfitting ranges

### Grid Search Cross Validation:

It is important to use grid search for hyper parameters tuning to reach the best hyper parmaters that lead to better model performance. This is very helpful as fro example when I try try SVR with C=1 an kernel rbf , it gives me about 40 % percent r2 , after applying Grid Search , it is found that the best parameters is C =10 with linear kernel with 70% which is higher by 30% 

In [58]:
model=SVR()
params = [
        {'C':[1, 10], 'kernel':['linear', 'sigmoid', 'poly']},
        {'C':[1, 10], 'kernel':['rbf'], 'gamma':[0.5, 0.6, 0.7, 0.1, 0.01, 0.01]}
         ]
grid_search_svc=GridSearchCV(estimator=model,
                        param_grid=params,
                        scoring='r2',
                        n_jobs=-1)

In [ ]:
grid_search_svc.fit(x_train,y_train)

In [11]:
grid_search_svc.best_params_

{'C': 10, 'kernel': 'linear'}

In [13]:
grid_search_svc.best_score_

0.7851789203542772

In [60]:
features=x.columns

In [61]:
pickle.dump(features,open("../models/features.h5",'wb'))
pickle.dump(scaler,open("../models/scaler.h5",'wb'))
pickle.dump(poly,open("../models/poly_features.h5",'wb'))